# HARD LEVEL

Даны два кольца: $\mathbb{Z}$ и $\mathbb{K}[x]$.  
Пусть в каждом из них взят конечно порождённый идеал. Нужно показать, что он главный, и описать критерий принадлежности элемента идеалу.

---

## Общий шаг: деление с остатком -> алгоритм Евклида -> НОД и Безу

**Деление с остатком в $\mathbb{Z}$.**  
Для любых $a,b\in\mathbb{Z}$, $b\ne 0$, существуют $q,r\in\mathbb{Z}$ такие что
$$
a=bq+r,\qquad 0\le r<|b|.
$$

**Деление с остатком в $\mathbb{K}[x]$.**  
Для любых $f(x),g(x)\in\mathbb{K}[x]$, $g(x)\ne 0$, существуют $q(x),r(x)\in\mathbb{K}[x]$ такие что
$$
f(x)=g(x)q(x)+r(x),\qquad \deg r(x)<\deg g(x).
$$

Из существования деления с остатком следует, что в обоих кольцах применим алгоритм Евклида. Он завершается, значит для любого конечного набора элементов существует НОД, и выполняется тождество Безу.

---

## Случай 1: $\mathbb{Z}$

Пусть $a_1,\dots,a_k\in\mathbb{Z}$ и $I=(a_1,\dots,a_k)\subseteq\mathbb{Z}$.

### 1) Гланвый идеал

Обозначим $d=\gcd(a_1,\dots,a_k)$, $d\ge 0$.  
По тождеству Безу существуют $u_1,\dots,u_k\in\mathbb{Z}$ такие что
$$
u_1a_1+\dots+u_ka_k=d.
$$
Левая часть лежит в $I$, значит $d\in I$, откуда $(d)\subseteq I$.

С другой стороны, так как $d\mid a_i$ для каждого $i$, то $a_i\in(d)$, следовательно $I\subseteq(d)$.

Итак, $I=(d)$.

### 2) Критерий принадлежности

Для любого $f\in\mathbb{Z}$ имеем
$$
f\in I \iff d\mid f.
$$

---

## Случай 2: $\mathbb{K}[x]$

Пусть $f_1(x),\dots,f_k(x)\in\mathbb{K}[x]$ и $I=(f_1,\dots,f_k)\subseteq\mathbb{K}[x]$.

### 1) Главный идеал

Обозначим $d(x)=\gcd(f_1(x),\dots,f_k(x))$ и выберем $d(x)$ моническим.  
По тождеству Безу существуют $u_1(x),\dots,u_k(x)\in\mathbb{K}[x]$ такие что
$$
u_1(x)f_1(x)+\dots+u_k(x)f_k(x)=d(x).
$$
Левая часть лежит в $I$, значит $d(x)\in I$, откуда $(d(x))\subseteq I$.

С другой стороны, так как $d(x)\mid f_i(x)$ для каждого $i$, то $f_i(x)\in(d(x))$, следовательно $I\subseteq(d(x))$.

Итак, $I=(d(x))$.

### 2) Критерий принадлежности

Для любого $g(x)\in\mathbb{K}[x]$ имеем
$$
g(x)\in I \iff d(x)\mid g(x).
$$

# Прототип класса для работы с кольцами:

In [5]:
from typing import Union, List, Tuple

EPS: float = 1e-12  # всё, что по модулю ≤ EPS, считаем нулём в полиноме


class RingElement:
  """
  Элемент кольца: либо целое из Z, либо полином из K[x].

  data:
    - int                 -> целое число (элемент Z);
    - list[float] [a0,..] -> полином a0 + a1*x + ... + an*x^n (элемент K[x]).
  """
  def __init__(self, data: Union[int, List[float]]):
    self.is_polynomial = isinstance(data, list)
    self.data = data

  def __repr__(self) -> str:
    """
    Короткая печать: целое как есть, полином — как сумма членов (от младших степеней).
    """
    if not self.is_polynomial:
      return str(self.data)
    terms: List[str] = []
    for i, c in enumerate(self.data):
      if abs(c) <= EPS:
        continue
      if i == 0:
        terms.append(f"{c}")
      elif i == 1:
        terms.append(f"{c}*x")
      else:
        terms.append(f"{c}*x^{i}")
    return " + ".join(terms) if terms else "0"


# Вспомогательные функции

def gcd_int(a: int, b: int) -> int:
  """
  Возвращает НОД двух целых чисел
  """
  a, b = abs(a), abs(b)
  while b != 0:
    a, b = b, a % b
  return a


def trim_poly(p: List[float]) -> List[float]:
  """
  Обрезает хвостовые нули и очень маленькие коэффициенты у полинома.
  Гарантирует, что у нулевого полинома результат ровно [0.0].
  """
  if not p:
    return [0.0]
  q = [0.0 if abs(c) <= EPS else float(c) for c in p]
  while len(q) > 1 and abs(q[-1]) <= EPS:
    q.pop()
  return q


def poly_divmod(a: List[float], b: List[float]) -> Tuple[List[float], List[float]]:
  """
  Делит многочлен a(x) на b(x) с остатком: a = b*q + r, deg r < deg b (или r = 0).
  Работает с вещественными коэффициентами; использует отсечение по EPS.
  """
  a = trim_poly(a)
  b = trim_poly(b)
  if b == [0.0]:
    raise ZeroDivisionError("Деление на нулевой полином.")
  da, db = len(a) - 1, len(b) - 1
  if da < db:
    return [0.0], a[:]

  q = [0.0] * (da - db + 1)
  r = a[:]
  lb = b[-1]

  while len(r) - 1 >= db and trim_poly(r) != [0.0]:
    r = trim_poly(r)
    dr = len(r) - 1
    coef = r[-1] / lb
    shift = dr - db
    q[shift] = coef
    # r ← r − coef * x^shift * b
    for i in range(db + 1):
      r[i + shift] -= coef * b[i]

  return trim_poly(q), trim_poly(r)


def make_monic(p: List[float]) -> List[float]:
  """
  Делает полином моническим (если он не нулевой): делит на старший коэффициент.
  """
  p = trim_poly(p)
  if p == [0.0]:
    return [0.0]
  lc = p[-1]
  return [c / lc for c in p]


def poly_gcd(a: List[float], b: List[float]) -> List[float]:
  """
  Возвращает монический НОД двух многочленов по алгоритму Евклида.
  """
  a = trim_poly(a)
  b = trim_poly(b)
  while b != [0.0]:
    _, r = poly_divmod(a, b)
    a, b = b, r
  return make_monic(a)


# Основная функция

def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
  """
  Возвращает порождающий главного идеала, заданного списком elements.
  • Для Z: НОД.
  • Для K[x]: монический НОД.
  Бросает исключение, если список пуст или смешаны типы (Z и K[x]).
  """
  if not elements:
    raise ValueError("Список элементов не должен быть пустым.")
  same_kind = all(e.is_polynomial == elements[0].is_polynomial for e in elements)
  if not same_kind:
    raise TypeError("Нужен однородный список: все Z или все K[x].")

  if elements[0].is_polynomial:
    # K[x]
    if len(elements) == 1:
      g = make_monic(elements[0].data)
    else:
      g = elements[0].data
      for e in elements[1:]:
        g = poly_gcd(g, e.data)
    return RingElement(g)
  else:
    # Z
    g = abs(int(elements[0].data))
    for e in elements[1:]:
      g = gcd_int(g, int(e.data))
    return RingElement(g)

In [6]:
# Z: I = (8,14,20) -> d = 2
print("I=(8,14,20) -> d =", gcd_ring_elements([RingElement(8), RingElement(14), RingElement(20)]))

# K[x]: I = (x^3 - x, x^2 - 1) -> d = x^2 - 1 (монический)
print("I=(x^3-x, x^2-1) -> d =",
      gcd_ring_elements([RingElement([0.0,-1.0,0.0,1.0]),  # x^3 - x
                         RingElement([-1.0,0.0,1.0])]))     # x^2 - 1

I=(8,14,20) -> d = 2
I=(x^3-x, x^2-1) -> d = -1.0 + 1.0*x^2


# EXPERT LEVEL

# Задача (Expert): доказать, что `lex` и `grlex` — порядки на мономах

Работаем в кольце $R=\mathbb{K}[x_1,\dots,x_n]$.
Моном записываем как $x^{\mathbf{a}}=x_1^{a_1}\cdots x_n^{a_n}$, где $\mathbf{a}=(a_1,\dots,a_n)\in\mathbb{N}^n$.

## Определение: порядок на мономах
Отношение $\prec$ на $\mathbb{N}^n$ называется **порядком на мономах**, если выполняются:

1. (**Транзитивность**) $\mathbf{a}\prec\mathbf{b}$ и $\mathbf{b}\prec\mathbf{c}\Rightarrow \mathbf{a}\prec\mathbf{c}$.
2. (**Линейность**) для любых $\mathbf{a},\mathbf{b}$ верно ровно одно из: $\mathbf{a}=\mathbf{b}$, $\mathbf{a}\prec\mathbf{b}$, $\mathbf{b}\prec\mathbf{a}$.
3. (**Совместимость с умножением**) $\mathbf{a}\prec\mathbf{b}\Rightarrow \mathbf{a}+\mathbf{c}\prec\mathbf{b}+\mathbf{c}$ для любого $\mathbf{c}\in\mathbb{N}^n$.

(Здесь $\mathbf{a}+\mathbf{c}$ — покоординатное сложение показателей; умножение мономов соответствует сложению показателей.)

---

## 1) Лексикографический порядок (`lex`)

Фиксируем порядок переменных $x_1\succ x_2\succ\cdots\succ x_n$.

**Определение.**
$$
\mathbf{a}\prec_{\mathrm{lex}}\mathbf{b}
\;\Longleftrightarrow\;
\text{первая позиция } i \text{ с } a_i\neq b_i \text{ удовлетворяет } a_i<b_i.
$$

**Проверка свойств.**

- **(1) Транзитивность.** Лексикографический порядок на $\mathbb{Z}^n$ транзитивен; на $\mathbb{N}^n$ тоже.
- **(2) Линейность.** Для любых $\mathbf{a},\mathbf{b}$ либо $\mathbf{a}=\mathbf{b}$, либо найдётся первая позиция $i$ с $a_i\neq b_i$; тогда либо $a_i<b_i$, либо $a_i>b_i$.
- **(3) Совместимость.** Пусть $i$ — первая позиция, где $a_i<b_i$. Для любого $\mathbf{c}$ первые $i-1$ координат у $\mathbf{a}+\mathbf{c}$ и $\mathbf{b}+\mathbf{c}$ совпадают, а на $i$-й $a_i+c_i<b_i+c_i$. Значит, $\mathbf{a}+\mathbf{c}\prec_{\mathrm{lex}}\mathbf{b}+\mathbf{c}$.

**Вывод.** `lex` — корректный порядок на мономах.

---

## 2) Порядок по степени с лексикографией (`grlex`)

Обозначим $|\mathbf{a}|=\sum_{j=1}^n a_j$ (общая степень монома).

**Определение.**
$$
\mathbf{a}\prec_{\mathrm{grlex}}\mathbf{b}
\;\Longleftrightarrow\;
\big(|\mathbf{a}|<|\mathbf{b}|\big)
\ \text{ или }\
\big(|\mathbf{a}|=|\mathbf{b}|\ \&\ \mathbf{a}\prec_{\mathrm{lex}}\mathbf{b}\big).
$$

**Проверка свойств.**

- **(1) Транзитивность.**
  Если $|\mathbf a|<|\mathbf b|<|\mathbf c|$, то $|\mathbf a|<|\mathbf c|$.  
  В слоях $\sum_{i=1}^{n} a_i=\mathrm{const}$ транзитивность обеспечивает `lex`.  
  Смешанные случаи (например, $|\mathbf a|<|\mathbf b|=|\mathbf c|$) также дают $|\mathbf a|<|\mathbf c|$.
- **(2) Линейность.**
  Для любых $\mathbf{a},\mathbf{b}$ либо $|\mathbf{a}|\neq|\mathbf{b}|$ (решает сумма степеней), либо $|\mathbf{a}|=|\mathbf{b}|$ (решает `lex`). Всегда сравнимы.
- **(3) Совместимость.**
  Если $|\mathbf{a}|<|\mathbf{b}|$, то $|\mathbf{a}+\mathbf{c}|=|\mathbf{a}|+|\mathbf{c}|<|\mathbf{b}|+|\mathbf{c}|=|\mathbf{b}+\mathbf{c}|$.
  Если $|\mathbf{a}|=|\mathbf{b}|$ и $\mathbf{a}\prec_{\mathrm{lex}}\mathbf{b}$, то $|\mathbf{a}+\mathbf{c}|=|\mathbf{b}+\mathbf{c}|$, а `lex` устойчив к прибавлению одинакового $\mathbf{c}$ (первая отличающаяся позиция и знак сравнения сохраняются). Следовательно, $\mathbf{a}+\mathbf{c}\prec_{\mathrm{grlex}}\mathbf{b}+\mathbf{c}$.

**Вывод.** `grlex` — корректный порядок на мономах.

# ОХОТА НА ВЕДЬМ

# Ответы на вопросы

**Шаг редукции.** Если $LM(g_i)\mid LT(f)$, то
$$
f \;\longrightarrow\; f-\frac{LT(f)}{LT(g_i)}\,g_i .
$$
Повторяем, пока шаг невозможен ни для одного $g_i$. Итог — остаток (нормальная форма) $NF_G(f)$.

---

## 1) Если получился $0$ — что это означает?

**Ответ.** $NF_G(f)=0 \;\Rightarrow\; f\in\langle G\rangle$.

**Доказательство.** При каждом шаге мы вычитаем кратный одному из $g_i$:
$$
f=f_0 \to f_1 \to \cdots \to f_s=0,\qquad f_{k+1}=f_k-q_k g_{i_k}.
$$
Инвариант: $\,f_k=f-\sum_{j=0}^{k-1}q_j g_{i_j}$. В конце $f_s=0$, значит
$$
f=\sum_{j=0}^{s-1}q_j g_{i_j}\in\langle G\rangle.\
$$

---

## 2) Если получился не $0$ — что это означает?

**Ответ.** Остаток просто **не редуцируем дальше**: ни один его член не делится ни одним $LM(g_i)$.
В частности,
$$
LT\!\big(NF_G(f)\big)\notin \langle LM(G)\rangle.
$$

**Доказательство.** Если бы какой-то член остатка делился $LM(g_i)$, можно было бы сделать ещё один шаг — противоречие определению нормальной формы.

---

## 3) Есть ли «порядочный подданный», который не пройдёт ритуал?

**Ответ.** Да. Бывает $f\in\langle G\rangle$, но $NF_G(f)\neq0$.

**Достаточно привести пример**: $f=y-1\in\langle xy-1,\;x-y\rangle$, однако редукция останавливается на $y-1\neq0$.

---

## 4) Зависит ли результат ритуала от порядка действий?

**Ответ.** В общем случае — **да**, остаток может зависеть от того, в какой последовательности применять редукторы.

**Короткий пример** (тот же `lex`, $x\succ y$). Пусть $G=\{xy-1,\;x-y\}$ и $f=xy$.
- Сначала по $xy-1$: $\,xy\to xy-1\cdot(xy-1)=1 \Rightarrow NF_G(f)=1$.
- Сначала по $x-y$: $\,xy\to xy-y(x-y)=y^2 \Rightarrow NF_G(f)=y^2$.
Остатки разные: $1$ и $y^2$.

---

## Почему редукция всегда заканчивается?

Каждый шаг **убивает** текущий $LT(f)$, новый $LM$ строго **меньше** по $\prec$.  
Порядок на мономах — хорошо-упорядоченный (нет бесконечной убывающей цепочки), поэтому бесконечно редуцировать невозможно — процесс **терминирует**.

# ПРОЗРЕНИЕ. ОРДЕН ГРЁБНЕРА

# Задача (Expert). Любой ли набор полиномов — базис Грёбнера?

**Вопрос.** Верно ли для любых $g_1,\dots,g_m$ и $I=\langle g_1,\dots,g_m\rangle$, что  
$$
in(I)=\big(in(g_1),\dots,in(g_m)\big)\ ?
$$

**Ответ:** **нет**, в общем случае это неверно.

---

## Контрпример (порядок **lex** с $x\succ y$ в $\mathbb{K}[x,y]$)

Положим
$$
G=\{\,g_1=xy-1,\; g_2=x-1\,\},
\qquad
I=\langle G\rangle .
$$

1. Покажем, что $f:=y-1\in I$:
$$
f=(xy-1)-y(x-1)=g_1-y\,g_2\in\langle g_1,g_2\rangle=I.
$$

2. Ведущие члены:
$$
in(g_1)=xy,\qquad in(g_2)=x,\qquad in(f)=y .
$$

3. Идеал, порождённый ведущими членами $G$:
$$
\big(in(g_1),in(g_2)\big)=(xy,x)=(x),
$$
он **не** содержит $y$.

4. Но так как $f\in I$, то $in(f)=y\in in(I)$. Следовательно,
$$
y\in in(I),\qquad y\notin (x)
\ \Longrightarrow\
\boxed{\,in(I)\neq (in(g_1),in(g_2))\,}.
$$

# ОТВЕТЫ НА ВОПРОСЫ

# Редукция по базису Грёбнера: ответы на вопросы

**Фиксация.** Выбираем порядок на мономах $\prec$. Пусть $G=\{g_1,\dots,g_m\}$ — базис Грёбнера идеала $I=\langle G\rangle\subseteq\mathbb{K}[x_1,\dots,x_n]$ относительно $\prec$.

**Обозначения.** Для ненулевого многочлена $f$:
- $LM(f)$ — ведущий моном (наибольший по $\prec$);
- $LC(f)$ — ведущий коэффициент;
- $LT(f)=LC(f)\,LM(f)$ — ведущий член;
- $NF_G(f)$ — нормальная форма (остаток) при делении $f$ по $G$.

---

## 1) Если получился $0$, что это означает?

**Утверждение.** $NF_G(f)=0\ \Longleftrightarrow\ f\in I$.

**Доказательство.** По алгоритму деления (редукции) всегда существует представление
$$
f=\sum_{i=1}^m q_i\,g_i + r,\qquad r=NF_G(f),
$$
где ни один моном из $r$ не делится ни одним $LM(g_i)$.

- Если $r=0$, то очевидно $f\in I$.
- Обратно, пусть $f\in I$. Тогда $r=f-\sum q_i g_i\in I$, следовательно $LT(r)\in in(I)$. Так как $G$ — базис Грёбнера, то
$$
in(I)=\big(LM(g_1),\dots,LM(g_m)\big).
$$
Значит $LT(r)$ делится некоторым $LM(g_j)$ — противоречие редуцированности $r$. Следовательно, $r=0$.

---

## 2) Если получился не $0$, что это означает?

**Утверждение.** Если $NF_G(f)\neq 0$, то $f\notin I$, а $NF_G(f)$ — нормальная форма (редуцированный остаток), зависящая только от $f$, от $G$ и от фиксированного порядка $\prec$.

**Доказательство (принадлежность).** Если $NF_G(f)=r\neq 0$, но $f\in I$, то, как выше, $r\in I$ и $LT(r)\in in(I)$, а значит $LT(r)$ делится некоторым $LM(g_j)$ — противоречие редуцированности $r$. Следовательно, $f\notin I$.

---

## 3) Существует ли порядочный подданный, который не сможет пройти ритуал?

**Ответ.** **Нет.** При редукции по базису Грёбнера всякий $f\in I$ редуцируется ровно в $0$.

**Доказательство.** Пусть $f\in I$ и $f=\sum q_i g_i + r$, где $r=NF_G(f)$ редуцирован. Из $r\in I$ следует $LT(r)\in in(I)=\big(LM(g_1),\dots,LM(g_m)\big)$, то есть $LT(r)$ делится некоторым $LM(g_j)$, что невозможно для редуцированного $r\neq 0$. Следовательно, $r=0$, т.е. $NF_G(f)=0$.

---

## 4) Зависит ли результат ритуала от того, в каком порядке проводить редукцию?

**Ответ.** **Нет.** Для базиса Грёбнера остаток $NF_G(f)$ **не зависит** от выбора редуктора на каждом шаге (результат единственен).

**Доказательство (уникальность нормальной формы).** Пусть две стратегии редукции дают
$$
f=\sum q_i g_i + r_1=\sum \tilde q_i g_i + r_2,
$$
где $r_1,r_2$ редуцированы по $G$. Тогда
$$
r_1-r_2=\sum (\tilde q_i-q_i)\,g_i\in I.
$$
Если $r_1\neq r_2$, то $LT(r_1-r_2)\in in(I)=\big(LM(g_1),\dots,LM(g_m)\big)$, то есть $LT(r_1-r_2)$ делится некоторым $LM(g_j)$. Но, поскольку $r_1$ и $r_2$ редуцированы, ведущий моном разности $LT(r_1-r_2)$ обязан совпадать с $LM(r_1)$ или с $LM(r_2)$ (он не может «спрятаться» за делением). Получаем, что у $r_1$ или $r_2$ есть моном, делящийся $LM(g_j)$ — противоречие редуцированности. Следовательно, $r_1=r_2$.

# Это не конец?

Любой базис Грёбнера можно привести к редуцированному

Пусть зафиксирован порядок на мономах $\prec$. Пусть
$G={g_1,\dots,g_m}$ — базис Грёбнера идеала $I=\langle G\rangle\subset \mathbb{K}[x_1,\dots,x_n]$ (относительно $\prec$).

Докажем, что из $G$ можно получить редуцированный базис Грёбнера того же идеала (для того же порядка).

---

Построение

Шаг 1 (сделать моническими).
Заменяем каждый $g_i$ на $g_i/LC(g_i)$. Это не меняет ни идеал $\langle G\rangle$, ни ведущие мономы $LM(g_i)$.

Шаг 2 (Убрать элементы, чей ведущий моном делится ведущим моном другого элемента).
Удаляем из $G$ все элементы, чей ведущий моном делится ведущим моном другого элемента. Обозначим результат через $H$.

Тогда $\langle LM(H)\rangle=\langle LM(G)\rangle$.
Следовательно, $H$ остаётся базисом Грёбнера того же идеала, поскольку
$$
\langle LM(G)\rangle=in(I)\quad\Longleftrightarrow\quad G\text{ — базис Грёбнера}.
$$
Шаг 3 (нормализовать хвосты).
Для каждого $h\in H$ редуцируем его по множеству $H\setminus{h}$ до остатка $r$ (запрещая деление ведущего монома самого $h$). Получаем множество
$$
G^{\mathrm{0}}=\{\,r\ \text{для всех}\ h\in H\,\}.
$$

---

### Почему это редуцированный базис

1. На шаге 1 все элементы сделали моническими; редукция хвостов на шаге 3 не меняет ведущий коэффициент, поэтому $\mathrm{LC}(r)=1$.

2. **Ведущие мономы не делятся попарно.** Шаг 2 обеспечивает это для $H$. На шаге 3 редуцируются только хвосты, значит $\mathrm{LM}(r)=\mathrm{LM}(h)$; следовательно, отношения делимости между лидерами сохраняются.

3. **Хвосты нормальны.** По построению на шаге 3 каждый хвост $\,r-\mathrm{LT}(r)\,$ не делится ни на один ведущий моном другого элемента, то есть ни на один $\mathrm{LM}(g_j)$.

4. **Это остаётся базисом Грёбнера того же идеала.** Переход $h\mapsto r$ не меняет лидера ($\mathrm{LM}(r)=\mathrm{LM}(h)$), поэтому начальный идеал сохраняется:
$$
\langle \mathrm{LM}(G^{\mathrm{0}})\rangle \;=\; \langle \mathrm{LM}(H)\rangle \;=\; \langle \mathrm{LM}(G)\rangle \;=\; \mathrm{in}(I).
$$
Значит, $G^{\mathrm{0}}$ — редуцированный базис Грёбнера того же идеала.


# ДЕЛА ГЕРОЯ

# Алгоритм Бухбергера с оптимизациями

In [7]:
from __future__ import annotations
from fractions import Fraction
from dataclasses import dataclass
from typing import Dict, Tuple, List, Iterable, Optional

Monom = Tuple[int, ...]  # (a1, a2, ..., an) — степени по x1,...,xn

# Вспомогательные функции

def mono_add(a: Monom, b: Monom) -> Monom:
  """Складывает показатели: x^a * x^b = x^(a+b)"""
  return tuple(ai + bi for ai, bi in zip(a, b))

def mono_sub(a: Monom, b: Monom) -> Monom:
  """Вычитает показатели: x^a / x^b = x^(a-b). Предполагаем b | a"""
  return tuple(ai - bi for ai, bi in zip(a, b))

def mono_leq(a: Monom, b: Monom) -> bool:
  """Покомпонентное ≤ (удобно для проверок)"""
  return all(ai <= bi for ai, bi in zip(a, b))

def mono_divides(a: Monom, b: Monom) -> bool:
  """Проверяет, делит ли моном a моном b (x^a | x^b)"""
  return mono_leq(a, b)

def mono_lcm(a: Monom, b: Monom) -> Monom:
  """LCM для мономов — максимум по координатам"""
  return tuple(max(ai, bi) for ai, bi in zip(a, b))

def mono_coprime(a: Monom, b: Monom) -> bool:
  """Взаимно простые мономы: ни по одной координате нет общей положительной степени"""
  return all(min(ai, bi) == 0 for ai, bi in zip(a, b))

def mono_deg(a: Monom) -> int:
  """Суммарная степень монома"""
  return sum(a)

def lex_key(m: Monom) -> Tuple[int, ...]:
  """Ключ для сортировки по lex (чем больше кортеж — тем «старше» моном)"""
  return m

def grlex_key(m: Monom) -> Tuple[int, ...]:
  """Ключ для сортировки по grlex: сначала сумма степеней, потом lex"""
  return (mono_deg(m),) + m

def order_key(name: str):
  """Возвращает функцию-ключ для сравнения мономов (max(..., key=...))"""
  if name == "lex":
    return lex_key
  if name == "grlex":
    return grlex_key
  raise ValueError("Параметр order должен быть ‘lex’ или ‘grlex’")

# Полиномы

@dataclass
class RingElement:
  """
  Полином над Q в K[x1,...,xn].
  coeffs: словарь {моном -> коэффициент}, нули не храним.
  n: число переменных.
  order: 'lex' или 'grlex'.
  """
  coeffs: Dict[Monom, Fraction]
  n: int
  order: str = "lex"

  # конструкторы и служебные
  @staticmethod
  def from_terms(terms: Dict[Monom, int | Fraction], n: int, order: str = "lex") -> "RingElement":
    """Удобно создать из словаря чисел: приведёт к Fraction и подчистит нули."""
    co: Dict[Monom, Fraction] = {}
    for m, c in terms.items():
      if len(m) != n:
        raise ValueError("Длина монома должна равняться n")
      cf = Fraction(c)
      if cf:
        co[m] = co.get(m, Fraction(0)) + cf
    co = {m: c for m, c in co.items() if c}
    return RingElement(co, n, order)

  def copy(self) -> "RingElement":
    return RingElement(dict(self.coeffs), self.n, self.order)

  def is_zero(self) -> bool:
    return not self.coeffs

  # строки
  def __repr__(self) -> str:
    if not self.coeffs:
      return "0"
    items = []
    for m, c in sorted(self.coeffs.items(), key=lambda kv: order_key(self.order)(kv[0]), reverse=True):
      mon = []
      for i, e in enumerate(m, start=1):
        if e == 0:
          continue
        if e == 1:
          mon.append(f"x{i}")
        else:
          mon.append(f"x{i}^{e}")
      mon_str = "*".join(mon) if mon else "1"
      if c == 1 and mon_str != "1":
        items.append(mon_str)
      elif c == -1 and mon_str != "1":
        items.append(f"-{mon_str}")
      else:
        items.append(f"{c}*{mon_str}")
    s = " + ".join(items)
    s = s.replace("+ -", "- ")
    return s

  # базовые операции
  def with_coeffs(self, d: Dict[Monom, Fraction]) -> "RingElement":
    return RingElement(d, self.n, self.order)

  def add(self, other: "RingElement") -> "RingElement":
    """Сумма полиномов"""
    if self.n != other.n or self.order != other.order:
      raise ValueError("incompatible polynomials")
    co = dict(self.coeffs)
    for m, c in other.coeffs.items():
      co[m] = co.get(m, Fraction(0)) + c
      if co[m] == 0:
        del co[m]
    return self.with_coeffs(co)

  def sub(self, other: "RingElement") -> "RingElement":
    """Разность полиномов"""
    return self.add(other.mul_scalar(Fraction(-1)))

  def mul_scalar(self, k: int | Fraction) -> "RingElement":
    """Умножение на скаляр из поля"""
    kk = Fraction(k)
    if kk == 0 or self.is_zero():
      return self.with_coeffs({})
    return self.with_coeffs({m: kk * c for m, c in self.coeffs.items()})

  def mul_monom(self, m: Monom, k: int | Fraction = 1) -> "RingElement":
    """Умножение на моном x^m и скаляр k"""
    kk = Fraction(k)
    if kk == 0 or self.is_zero():
      return self.with_coeffs({})
    return self.with_coeffs({mono_add(mm, m): kk * c for mm, c in self.coeffs.items()})

  # ведущий моном (LM) и ведущий член (LT)
  def leading_monomial(self) -> Optional[Monom]:
    """LM(f): старший моном по заданному порядку или None для нуля"""
    if not self.coeffs:
      return None
    key = order_key(self.order)
    return max(self.coeffs.keys(), key=key)

  def leading_term(self) -> Optional[Tuple[Monom, Fraction]]:
    """LT(f) = LC(f)*LM(f). Возвращает (LM, LC) или None для нуля"""
    lm = self.leading_monomial()
    if lm is None:
      return None
    return lm, self.coeffs[lm]

  def make_monic(self) -> "RingElement":
    """Сделать полином моническим (если нулевой — вернуть как есть)"""
    lt = self.leading_term()
    if lt is None:
      return self
    lm, lc = lt
    return self.mul_scalar(Fraction(1, 1) / lc)

# Деление / Нормальная форма / S-полином

def normal_form(f: RingElement, G: Iterable[RingElement]) -> RingElement:
  """
  Деление f по набору G до остатка (Нормальная форма).
  Берём текущий старший моном r и пробуем поделить его LM-ами из G.
  """
  r = f.copy()
  if r.is_zero():
    return r
  # всегда выбирать полином-делитель по фиксированному правилу (LM по order, tie-break: индекс)
  divs = [g for g in G if not g.is_zero()]
  divs.sort(key=lambda g: order_key(g.order)(g.leading_monomial()), reverse=True)

  while True:
    lt = r.leading_term()
    if lt is None:
      break
    lm_r, lc_r = lt
    reduced = False
    for g in divs:
      lm_g, lc_g = g.leading_term()
      if mono_divides(lm_g, lm_r):
        # коэффициент и моном, которыми нужно умножить g, чтобы снять LT(r)
        m = mono_sub(lm_r, lm_g)
        factor = lc_r / lc_g
        r = r.sub(g.mul_monom(m, factor))
        reduced = True
        break
    if not reduced:
      # Не нашлось делителя для LM — зафиксируем этот член и снимем его вручную.
      # Перенесём LT в остаток и вычтем его (эквивалентно добавлению нулевого делителя).
      lm, lc = r.leading_term()
      # фиксируем ведущий и отнимаем его (так мы сдвинем старший моном вниз)
      r = r.sub(RingElement({lm: lc}, r.n, r.order))
  return r

def S_polynomial(f: RingElement, g: RingElement) -> RingElement:
  """
  S(f,g) = (x^lcm / LT(f)) * f - (x^lcm / LT(g)) * g, где деление по коэффициенту тоже в поле.
  """
  lt_f = f.leading_term()
  lt_g = g.leading_term()
  if lt_f is None or lt_g is None:
    return RingElement({}, f.n, f.order)
  lm_f, lc_f = lt_f
  lm_g, lc_g = lt_g
  l = mono_lcm(lm_f, lm_g)
  mf = mono_sub(l, lm_f)
  mg = mono_sub(l, lm_g)
  term_f = f.mul_monom(mf, Fraction(1, 1) / lc_f)
  term_g = g.mul_monom(mg, Fraction(1, 1) / lc_g)
  return term_f.sub(term_g)

# Бухбергер + редукция

def buchberger(F: List[RingElement], use_product_criterion: bool = True) -> List[RingElement]:
  """
  Алгоритм Бухбергера. На выходе — редуцированный монический базис Грёбнера
  для того же идеала и того же порядка на мономах.
  """
  if not F:
    return []

  # привести вход к моническим
  G: List[RingElement] = [f.make_monic() for f in F if not f.is_zero()]

  # основной цикл по парам
  pairs = {(i, j) for i in range(len(G)) for j in range(i + 1, len(G))}
  while pairs:
    i, j = pairs.pop()
    fi, fj = G[i], G[j]

    # Критерий: если ведущие мономы LM(f_i) и LM(f_j) взаимно просты,
    # S(f_i,f_j) редуцируется в 0 по текущему G -> пару пропускаем.
    if use_product_criterion and mono_coprime(fi.leading_monomial(), fj.leading_monomial()):
      continue

    s = S_polynomial(fi, fj)
    r = normal_form(s, G)
    if not r.is_zero():
      r = r.make_monic()
      # добавить в базис и все пары с ним
      G.append(r)
      k = len(G) - 1
      for t in range(k):
        if t != k:
          pairs.add((t, k))

  # Удаляем элементы, чьи ведущие мономы делятся ведущими мономами других
  keep: List[RingElement] = []
  for i, g in enumerate(G):
    lm_g = g.leading_monomial()
    drop = False
    for j, h in enumerate(G):
      if i == j:
        continue
      if mono_divides(h.leading_monomial(), lm_g):
        drop = True
        break
    if not drop:
      keep.append(g)
  G = keep

  # редукция хвостов (получаем редуцированный базис)
  Gred: List[RingElement] = []
  for i, g in enumerate(G):
    lm, lc = g.leading_term()
    tail = g.sub(RingElement({lm: lc}, g.n, g.order))
    others = [G[j] for j in range(len(G)) if j != i]
    r_tail = normal_form(tail, others)
    grev = RingElement({lm: Fraction(1)} | r_tail.coeffs, g.n, g.order).make_monic()
    Gred.append(grev)

  # Отсортируем по LM в фиксированном порядке
  Gred.sort(key=lambda p: order_key(p.order)(p.leading_monomial()), reverse=True)
  return Gred

# Функция проверки полинома на верность королю

In [19]:
# Проверка в идеале

def is_in_ideal(f: RingElement, Gred: List[RingElement]) -> bool:
  """
  Проверяет принадлежность: f ∈ <Gred>, где Gred — редуцированный базис Грёбнера.
  Просто считаем NF_Gred(f) и смотрим, 0 ли.
  """
  return normal_form(f, Gred).is_zero()

# Тесты

def poly(terms: Dict[Monom, int | Fraction], n: int = 2, order: str = "lex") -> RingElement:
  """Короткий конструктор для тестов: poly({(1,0):1,(0,1):-1}) = x1 - x2"""
  return RingElement.from_terms(terms, n=n, order=order)

# Примеры (K=Q, переменные x1,x2, порядок lex: x1 ≻ x2)
x, y = (1, 0), (0, 1)

# 1) Базовый: G = {xy-1, x-y}. Базис и проверка примера из теории.
G0 = [poly({mono_add(x, y): 1, (0, 0): -1}, order="lex"),
      poly({x: 1, y: -1}, order="lex")]

Gred = buchberger(G0)
print("Gred (lex):")
for g in Gred:
  print("  ", g)

f = poly({y: 1, (0, 0): -1}, order="lex")  # y - 1
print("NF_G(y-1) =", normal_form(f, Gred))
print("is_in_ideal(y-1, Gred)?", is_in_ideal(f, Gred))

# 2) Пример: G={x^2 - y, xy - 1} (lex). Проверка принадлежности полинома f идеалу G и норм формы.
G1 = [poly({(2,0): 1, (0,1): -1}, order="lex"),
      poly({mono_add(x, y): 1, (0, 0): -1}, order="lex")]
G1red = buchberger(G1)
print("\nG1red (lex):")
for g in G1red:
  print("  ", g)

f2 = poly({(3,0): 1}, order="lex")  # x^3
print("NF_G1(x^3) =", normal_form(f2, G1red))
print("in ideal?", is_in_ideal(f2, G1red))
print("in ideal?", is_in_ideal(f2, G1red))

Gred (lex):
   x1
NF_G(y-1) = 0
is_in_ideal(y-1, Gred)? True

G1red (lex):
   x1^2
   x1*x2
NF_G1(x^3) = 0
in ideal? True
in ideal? True


# Практические применения базиса Грёбнера

In [9]:
!pip -q install sympy

In [10]:
from sympy import symbols, groebner, solve, factor, simplify

In [11]:
def verifies(F, sol):
    """Все ли исходные уравнения обращаются в 0 на данном решении sol (dict)?"""
    return all(simplify(p.subs(sol)) == 0 for p in F)

def as_set_of_tuples(solutions, gens):
    """Нормализуем множество решений к множеству упорядоченных кортежей по gens."""
    return {tuple(s[g] for g in gens) for s in solutions}

Для задачи исключения переменных удобно брать лексикографический порядок lex с приоритетом, например, x $\succ y \succ z$. Для lex базис Грёбнера автоматически даёт треугольную систему: появляются полиномы только от последних переменных (напр., только от y или только от z), что и есть исключение переменных.

Пример 1: окружность и прямая

\begin{cases}
x^2 + y^2 - 1 = 0,\\
x - y = 0.
\end{cases}

$$
x=y\Rightarrow 2y^2-1=0\Rightarrow y=\pm \frac{1}{\sqrt{2}}, x=y.
$$

In [12]:
x, y = symbols('x y')

F = [x**2 + y**2 - 1, x - y]

# Лексикографический порядок: x ≻ y
G_lex = groebner(F, x, y, order='lex')
print("Groebner basis (lex):")
for p in G_lex.polys:
    print("  ", p.as_expr())

# Полиномы, зависящие только от y (исключили x)
only_y = [p.as_expr() for p in G_lex.polys if p.as_expr().free_symbols <= {y}]
print("\nИсключённые уравнения по y:", only_y)

# Решение по y, затем x=y
sol_y = solve(only_y[0], y)
sol_xy = [{x: s, y: s} for s in sol_y]
print("\nРешения:", sol_xy)

Groebner basis (lex):
   x - y
   2*y**2 - 1

Исключённые уравнения по y: [2*y**2 - 1]

Решения: [{x: -sqrt(2)/2, y: -sqrt(2)/2}, {x: sqrt(2)/2, y: sqrt(2)/2}]


In [13]:
# Сравнение с CAS для Примера 1
gens = (x, y)              # порядок переменных
sol_gb  = sol_xy           # решения через базис Грёбнера
sol_all = solve(F, gens, dict=True)

print("Каждое sol_gb обнуляет F? ->", [verifies(F, s) for s in sol_gb])
print("Совпадают множества решений? ->",
      as_set_of_tuples(sol_gb, gens) == as_set_of_tuples(sol_all, gens))

Каждое sol_gb обнуляет F? -> [True, True]
Совпадают множества решений? -> True


Пример 2: гипербола и прямая

\begin{cases}
xy - 1 = 0,\\
x - y   = 0.
\end{cases}

$$
x=y\Rightarrow x^2=1 \Rightarrow x=y=\pm 1.
$$

In [14]:
x, y = symbols('x y')

F = [x*y - 1, x - y]
G_lex = groebner(F, x, y, order='lex')
print("Groebner basis (lex):")
for p in G_lex.polys:
    print("  ", p.as_expr())

only_y = [p.as_expr() for p in G_lex.polys if p.as_expr().free_symbols <= {y}]
print("\nИсключённые уравнения по y:", only_y)

sol_y = solve(only_y[0], y)
sol_xy = [{x: s, y: s} for s in sol_y]
print("\nРешения:", sol_xy)

Groebner basis (lex):
   x - y
   y**2 - 1

Исключённые уравнения по y: [y**2 - 1]

Решения: [{x: -1, y: -1}, {x: 1, y: 1}]


In [15]:
# Сравнение с CAS для Примера 2
gens = (x, y)
sol_gb  = sol_xy
sol_all = solve(F, gens, dict=True)

print("Каждое sol_gb обнуляет F? ->", [verifies(F, s) for s in sol_gb])
print("Совпадают множества решений? ->",
      as_set_of_tuples(sol_gb, gens) == as_set_of_tuples(sol_all, gens))

Каждое sol_gb обнуляет F? -> [True, True]
Совпадают множества решений? -> True


Пример 3: явное исключение до уравнения в одной переменной

\begin{cases}
x + y + z - 1 = 0,\\
x - z = 0,\\
y - z^2 = 0.
\end{cases}

Из первых двух x=z, из третьего y=z^2. Подстановка в первое: z^2 + 2z - 1 = 0.

In [16]:
x, y, z = symbols('x y z')

F = [x + y + z - 1, x - z, y - z**2]
G_lex = groebner(F, x, y, z, order='lex')  # x ≻ y ≻ z
print("Groebner basis (lex):")
for p in G_lex.polys:
    print("  ", p.as_expr())

# Полиномы только от z — исключили x,y
only_z = [p.as_expr() for p in G_lex.polys if p.as_expr().free_symbols <= {z}]
print("\nУравнение по z:", only_z)

# Решаем по z и восстанавливаем x,y
sol_z = solve(only_z[0], z)
sol_xyz = [{z: s, x: s, y: s**2} for s in sol_z]
print("\nРешения:", [ {v: factor(val) for v, val in d.items()} for d in sol_xyz ])

Groebner basis (lex):
   x - z
   y + 2*z - 1
   z**2 + 2*z - 1

Уравнение по z: [z**2 + 2*z - 1]

Решения: [{z: -1 + sqrt(2), x: -1 + sqrt(2), y: (-1 + sqrt(2))**2}, {z: -sqrt(2) - 1, x: -sqrt(2) - 1, y: (1 + sqrt(2))**2}]


Для lex часто растут промежуточные коэффициенты, зато базис даёт «треугольную» форму (исключение переменных). grlex обычно устойчивее по размеру промежуточных полиномов, но не гарантирует исключение.
В алгоритме Бухбергера ускорение дают:

1. Критерий произведения и цепной критерий — пропускают пары
S-полиномов, которые гарантированно редуцируются в ноль;
2. Редукция до редуцированного базиса;
3. Стратегия выбора пар;
4. Модульные методы и современные варианты F4/F5.